# Esercizio 2 - Content2Word

L'obiettivo di questo erercizio è, date le definizioni presenti nel file *def.csv* usate per l'esercizio 1, risalire al termine che le descrive. Si tratta quindi di una ***ricerca onomasiologica***.

### Approccio

- Innanzitutto vengono presi i termini più frequenti nelle definizioni, che saranno i ***genus***, usati per restringere la ricerca su *WordNet*.

- ***Text cleaning***: rimozione di *stopwords* e *lemmatizzazione*.
- Prelievo di tutto il sottoalbero di iponimi dei genus e salvataggio delle loro definizioni (glosse).
- Confronto tra le definizioni di *Wordnet* e la nostra lista di definizioni. Il termine scelto è quello che massimizza l'*overlap* tra la definizione di *Wordnet* e la lista di definizioni in *def.csv*.
- Infine viene restituito il synset che ha la definizione più simile a quella della lista

In [29]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import random

### Funzioni

In [30]:
def get_text_from_file(path): # rimuove le stopwords e restituisce lista di parole nel file in path
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def get_most_freq_words(text, nword): # restituisce le n parole più frequenti in text, per ogni riga del documento
    genus = []
    for row in text:
        c = Counter()
        c.update(row)
        genus.append(c.most_common(nword))
    return genus

def get_hypos(word): # restituisce tutti gli iponimi di word
    syn = get_synset(word)
    hypo_list = []
    if(syn is not None):
        hypo_list = list(set([w for s in syn.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    return hypo_list

def get_hypers(word): # restituisce tutti gli iperonimi di word
    syn = get_synset(word)
    hyper_list = []
    if(syn is not None):
        hyper_list = list(set([w for s in syn.closure(lambda s:s.hypermyms()) for w in s.lemma_names()]))
    return hyper_list

def get_synset(word): # restituisce il primo synset di word
    if(len(wn.synsets(word)) > 0):
        return wn.synsets(word)[0]
    return None


### Esecuzione

In [31]:
num_genus = 3 
num_most_freq_word = 10 
starting_words = ["Emotion", "Person", "Revenge", "Brick"]

In [32]:
file = get_text_from_file('../data/def.csv')
genus = get_most_freq_words(file, num_genus)
print (genus)

[[('feeling', 11), ('human', 8), ('feel', 8)], [('human', 26), ('person', 5), ('homo', 5)], [('someone', 14), ('feeling', 7), ('anger', 7)], [('used', 22), ('object', 15), ('material', 13)]]


In [37]:
key_words_defs = get_most_freq_words(file, num_most_freq_word) # 10 parole più frequenti per ogni riga del documento

for i in range(len(genus)):
    
    # collasso key_words_defs in un unico array
    key_row = []
    for el in key_words_defs[i]:
        key_row.append(el[0])

    # salvo gli iponimi di ogni genus
    hypo_list, hypo_def = [], []
    for el in genus[i]:
        hypo_list.append(get_hypos(el[0])) 
     
    hypo_list = [x for xs in hypo_list for x in xs]

    # definizioni di ogni iponimo
    for hypo in hypo_list:
        hypo_def.append((hypo, get_synset(hypo).definition()))
    
    # salvo parole che massimizzano l'overlap e i punteggi (numero di intersezioni)
    res = []
    for wndef in hypo_def: # definizioni di ogni iponimo 
        score = 0
        imp_words = []
        for key_word in key_row: # definizioni in def.csv
            if(key_word in wndef[1]):
                score += 1
                imp_words.append(key_word)      

        res.append((score, wndef[0], imp_words, wndef[1]))
        
    # ordino sullo score e inverto
    sorted_list = sorted(res, key=lambda x: x[0]) 
    sorted_res = list(reversed(sorted_list))
    
    print("\nOriginal Word:", starting_words[i], "/  Genus -->",genus[i][0])
    for k in range(min(len(sorted_res), 5)):
        # print(f'Word: *{sorted_res[k][1]}*, score: *{sorted_res[k][0]}*, the key words are *{sorted_res[k][2]}* and the definition is *{sorted_res[k][3]}*')
        print(f'Word: *{sorted_res[k][1]}*, score: *{sorted_res[k][0]}*')


Original Word: Emotion /  Genus --> ('feeling', 11)
Word: *affect*, score: *3*
Word: *eagerness*, score: *3*
Word: *ambivalency*, score: *3*
Word: *daze*, score: *3*
Word: *discomfort*, score: *3*

Original Word: Person /  Genus --> ('human', 26)
Word: *Zuni*, score: *2*
Word: *Algonquian*, score: *2*
Word: *Elamite*, score: *2*
Word: *Cocopa*, score: *2*
Word: *Kanarese*, score: *2*

Original Word: Revenge /  Genus --> ('someone', 14)
Word: *sounding_board*, score: *5*
Word: *impassiveness*, score: *4*
Word: *unemotionality*, score: *4*
Word: *emotionlessness*, score: *4*
Word: *hatred*, score: *4*

Original Word: Brick /  Genus --> ('used', 22)
Word: *brick*, score: *5*
Word: *building_material*, score: *4*
Word: *marble*, score: *3*
Word: *kaolin*, score: *3*
Word: *Plasticine*, score: *3*


## Analisi dei risultati

Alcuni problemi riscontrati:

- **I genus**: i genus di riferimento hanno poco a che vedere con i termini, come ad esempio *"someone"* per *revenge*. Questo è portato dal dataset in input. Si potrebbe ripulire il dataset rimuovendo le parole che non hanno a che vedere con il termine originale, ma così facendo si andrebbe a compromettere l'esercizio e si renderebbe questo approccio poco scalabile su altre basi di dati.
   
- **Iponimi**: non è detto che il termine ricercato sia un iponimo dei genus che selezioanti, infatti potrebbe essere 
   un iperonimo o essere proprio in un punto completamente diverso dell'albero di *Wordnet*. Un possibile miglioramento dell'algoritmo potrebbe
   essere quello di andare a prelevare altri synset oltre agli iponimi del genus, ad esempio andando a prelevare anche i fratelli del genus,
   senza allontarsi troppo per non far esplodere la complessità dell'algoritmo.

- **Funzione di similarità**: l'algoritmo è basato su uno *score* che corrisponde a quante parole simili ci sono nelle definizioni. Si potrebbe 
   incrementare lo *score* sulla base di altri fattori, come la funzione di similarità di wordnet *path_similarity*.
